In [33]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import warnings
warnings.filterwarnings('ignore')
#MIGHT HAVE TO ENCODE THE BINNING OF THE LENGTHS SO THIS CAN BE A 
#PARAMETER FOR A COLD START

In [34]:
US_trails = pd.read_csv('../US_trails_engineered.csv')

In [35]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'state', 'length_range'],
      dtype='object')

In [36]:
US_trails.dtypes

name                   object
location               object
difficulty             object
length                float64
ascent                  int64
descent                 int64
stars                 float64
latitude              float64
longitude             float64
summary                object
url                    object
difficulty_encoded      int64
category               object
type_Featured Ride      int64
type_Trail              int64
state                  object
length_range           object
dtype: object

In [37]:
#I don't need the latitude and longitude columns since I'll be filtering on those
columns_to_scale = list(US_trails.select_dtypes(exclude=['object']).columns.difference(['latitude', 'longitude']))
columns_to_scale

['ascent',
 'descent',
 'difficulty_encoded',
 'length',
 'stars',
 'type_Featured Ride',
 'type_Trail']

In [25]:
#this scales each column individually, BUT REALLY WHAT IS IT DOING?
columns_to_scale = []
for col in columns_to_scale:
    US_trails[col + "_scaled"] = preprocessing.scale(US_trails[col])

In [26]:
'''
#THIS DOES THE SAME AS ABOVE, EXCEPT THAT IT REWRITES ALL THE QUANTITATIVE COLUMNS WITH THEIR SCALED VALUES
scaler = preprocessing.StandardScaler()
scaler.fit(US_trails[quantitative_columns])
US_trails[quantitative_columns] = scaler.transform(US_trails[quantitative_columns])
US_trails.head()
'''

'\n#THIS DOES THE SAME AS ABOVE, EXCEPT THAT IT REWRITES ALL THE QUANTITATIVE COLUMNS WITH THEIR SCALED VALUES\nscaler = preprocessing.StandardScaler()\nscaler.fit(US_trails[quantitative_columns])\nUS_trails[quantitative_columns] = scaler.transform(US_trails[quantitative_columns])\nUS_trails.head()\n'

In [27]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'state', 'length_range', 'ascent_scaled', 'descent_scaled',
       'difficulty_encoded_scaled', 'length_scaled', 'stars_scaled',
       'type_Featured Ride_scaled', 'type_Trail_scaled'],
      dtype='object')

In [28]:
scaled_cols = [col for col in US_trails.columns if 'scaled' in col]
scaled_cols
#US_trails_scaled = US_trails.filter(regex='scaled')
#US_trails_scaled.head()

['ascent_scaled',
 'descent_scaled',
 'difficulty_encoded_scaled',
 'length_scaled',
 'stars_scaled',
 'type_Featured Ride_scaled',
 'type_Trail_scaled']

In [29]:
Big_X = US_trails[scaled_cols].values
Big_X

array([[ 2.948716  , -2.79569706,  2.40444594, ...,  1.47345507,
         2.6075075 , -2.6075075 ],
       [-0.09257181,  0.12246069,  1.6195909 , ...,  0.44678903,
         2.6075075 , -2.6075075 ],
       [-0.1825315 ,  0.20147031, -0.7349742 , ..., -0.57987702,
         2.6075075 , -2.6075075 ],
       ..., 
       [ 0.86338537, -0.22996377, -0.7349742 , ..., -0.57987702,
        -0.38350801,  0.38350801],
       [-0.12617121, -0.05635054, -0.7349742 , ..., -0.57987702,
        -0.38350801,  0.38350801],
       [ 2.32333359, -1.441098  ,  0.04988084, ..., -0.57987702,
        -0.38350801,  0.38350801]])

In [40]:
#this function will take in a state you ridden in, a trail you rode on, and the state you want to ride in and recommend
def cos_sim_recommendations(trail_name, state_name, scaled_features, n = 5, desired_state = None, desired_location= None):
    index = US_trails.index[(US_trails['name'] == trail_name) & (US_trails['state']==state_name)][0]
    #index is the index of the given trail in the main df based on it's name and state
    trail = scaled_features[index].reshape(1,-1)
    #trail is the scaled features of the given trail reshaped into a 1 row array
    cs = cosine_similarity(trail, scaled_features)
    #cs is the array of cosine similarity measurements of the trail compared to the scaled_features of the other trails
    rec_index = np.argsort(cs)[0][::-1][1:]
    #rec_index is an array of the indices of the cosine similarity measurements ordered by greatest to least, leaving out the given trail
    ordered_df = US_trails.loc[rec_index]
    #ordered_df gives the trails of US_trails ordered by the trails most similar to least similar
    if desired_state:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
        #if a desired state to ride in is given, the dataframe is returned with trails only in that state
    if desired_location:
        ordered_df = ordered_df[ordered_df['location']== desired_location]
        #if a desired location to ride in is given, the dataframe is returned with trails only in that location
    rec_df = ordered_df.head(n)
    #rec_df is a dataframe with the number of desired trails to be recommended
    orig_row = US_trails.loc[[index]].rename(lambda x: 'original')
    #orig_row renames the index of the original trail given as 'original'
    total = pd.concat([orig_row, rec_df])
    #this combines the original row labeled as original row and the n recommended trails that are most similar to the original
    return total
    

In [41]:
cos_sim_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X, n = 5, desired_state = 'Colorado', desired_location = 'Durango')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Mount Falcon and Lair O' the Bear Loop,Morrison,Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,https://www.mtbproject.com/trail/700918/mount-...,4,Featured Ride,1,0,Colorado,15-20
6224,Grandview Ridge & Horse Gulch Loop,Durango,Blue/Black,16.1,1822,-1830,4.0,37.2240,-107.8564,"Grandview Ridge, Horse Gulch, Animas River - s...",https://www.mtbproject.com/trail/6178414/grand...,4,Featured Ride,1,0,Colorado,15-20
6801,Raiders Ridge Tech Loop,Durango,Black,8.9,1376,-1360,3.7,37.2731,-107.8738,A great loop in Durango featuring some of Hors...,https://www.mtbproject.com/trail/5336753/raide...,5,Featured Ride,1,0,Colorado,5-10
6803,Animas Mountain Loop,Durango,Black,5.7,1492,-1494,3.3,37.3030,-107.8728,A fun and rocky singletrack climb and descent ...,https://www.mtbproject.com/trail/3498783/anima...,5,Featured Ride,1,0,Colorado,5-10
6225,Sale Barn - South Rim - Carbon Junction,Durango,Blue,7.2,752,-766,3.5,37.2238,-107.8563,Excellent introduction to Durango singletrack ...,https://www.mtbproject.com/trail/4290112/sale-...,3,Featured Ride,1,0,Colorado,5-10
6856,Missionary Ridge,Durango,Blue/Black,7.0,1452,-699,3.5,37.3525,-107.8027,A rough and rugged ride along Missionary Ridge...,https://www.mtbproject.com/trail/7005827/missi...,4,Trail,0,1,Colorado,5-10


In [42]:
cos_sim_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X, n = 5, desired_state = 'Colorado', desired_location = 'Salida')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Mount Falcon and Lair O' the Bear Loop,Morrison,Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,https://www.mtbproject.com/trail/700918/mount-...,4,Featured Ride,1,0,Colorado,15-20
8380,Cottonwood Tour,Salida,Blue/Black,23.1,2540,-2539,4.2,38.5381,-105.9901,This is a great way to ride the Cottonwood Tra...,https://www.mtbproject.com/trail/6309603/cotto...,4,Featured Ride,1,0,Colorado,20-25
8366,Monarch Crest to Agate Creek Loop,Salida,Black,36.6,4432,-4417,4.0,38.4383,-106.3838,A long dirt road climb to access the classic M...,https://www.mtbproject.com/trail/7017214/monar...,5,Featured Ride,1,0,Colorado,30-50
8383,Hwy 285/Mears Junction - CR 47YY/Silver Creek ...,Salida,Blue/Black,19.6,1437,-1396,4.0,38.4449,-106.1052,Another option to enjoy this section of the Ra...,https://www.mtbproject.com/trail/7027033/hwy-2...,4,Featured Ride,1,0,Colorado,15-20
8381,Rainbow Trail #1336: Hayden Creek to Methodist...,Salida,Black,31.8,6636,-5729,4.5,38.3300,-105.8171,A point-to-point route with outstanding views ...,https://www.mtbproject.com/trail/7018071/rainb...,5,Featured Ride,1,0,Colorado,30-50
8363,Monarch to Fooses Creek,Salida,Black,13.7,786,-3280,4.4,38.4960,-106.3247,Beautiful views on the start to the Monarch Cr...,https://www.mtbproject.com/trail/5838799/monar...,5,Featured Ride,1,0,Colorado,10-15


In [43]:
#this function will take in a state you ridden in, a trail you rode on, and the state you want to ride in and recommend
def euclidean_dist_recommendations(trail_name, state_name, scaled_features, n = 5, desired_state = None, desired_location= None):
    index = US_trails.index[(US_trails['name'] == trail_name) & (US_trails['state']==state_name)][0]
    #index is the index of the given trail in the main df based on it's name and state
    trail = scaled_features[index].reshape(1,-1)
    #trail is the scaled features of the given trail reshaped into a 1 row array
    cs = euclidean_distances(trail, scaled_features)
    #cs is the array of euclidean distance measurements of the trail compared to the scaled_features of the other trails
    rec_index = np.argsort(cs)[0][::-1][1:]
    #rec_index is an array of the indices of the euclidean distance measurements ordered by greatest to least, leaving out the given trail
    ordered_df = US_trails.loc[rec_index]
    #ordered_df gives the trails of US_trails ordered by the trails most similar to least similar
    if desired_state:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
        #if a desired state to ride in is given, the dataframe is returned with trails only in that state
    if desired_location:
        ordered_df = ordered_df[ordered_df['location']== desired_location]
        #if a desired location to ride in is given, the dataframe is returned with trails only in that location
    rec_df = ordered_df.head(n)
    #rec_df is a dataframe with the number of desired trails to be recommended
    orig_row = US_trails.loc[[index]].rename(lambda x: 'original')
    #orig_row renames the index of the original trail given as 'original'
    total = pd.concat([orig_row, rec_df])
    #this combines the original row labeled as original row and the n recommended trails that are most similar to the original
    return total

In [44]:
#THIS IS SOOOO MUCH WORSE!!!!! WHY????
euclidean_dist_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X, n = 5, desired_state = 'Colorado', desired_location = 'Salida')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Mount Falcon and Lair O' the Bear Loop,Morrison,Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,https://www.mtbproject.com/trail/700918/mount-...,4,Featured Ride,1,0,Colorado,15-20
8373,Colorado 14ers Loop: Segment 1 - Salida to Lea...,Salida,Blue,139.1,17227,-14120,5.0,38.5357,-105.9952,The western half of the route including the Co...,https://www.mtbproject.com/trail/7025816/color...,3,Trail,0,1,Colorado,100+
8499,County Rd 101,Salida,Green/Blue,1.0,233,0,0.0,38.5051,-105.9522,Needs Summary,https://www.mtbproject.com/trail/7025759/count...,2,Trail,0,1,Colorado,0-5
8498,County Rd 49,Salida,Green/Blue,4.9,1804,-11,0.0,38.4926,-105.9513,Needs Summary,https://www.mtbproject.com/trail/7025760/count...,2,Trail,0,1,Colorado,0-5
8485,Poblano,Salida,Green,0.2,33,-11,3.0,38.5438,-105.9916,A short beginner trail for someone new to moun...,https://www.mtbproject.com/trail/5129486/poblano,1,Trail,0,1,Colorado,0-5
8477,Sgt Pepper,Salida,Green,0.2,8,-61,3.0,38.5438,-105.9916,Cut your teeth on one of the shorter beginner ...,https://www.mtbproject.com/trail/5129563/sgt-p...,1,Trail,0,1,Colorado,0-5


In [45]:
cos_sim_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X, n = 5, desired_state = 'Utah', desired_location = 'Moab')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Mount Falcon and Lair O' the Bear Loop,Morrison,Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,https://www.mtbproject.com/trail/700918/mount-...,4,Featured Ride,1,0,Colorado,15-20
8225,Flat Pass,Moab,Blue/Black,18.2,2006,-2005,3.4,38.4763,-109.4220,"A neglected classic, running through spectacul...",https://www.mtbproject.com/trail/4643331/flat-...,4,Featured Ride,1,0,Utah,15-20
9197,Zion Curtain,Moab,Blue/Black,17.9,1676,-1677,3.5,39.1956,-109.0513,A loop ride to a stunning Rabbit Valley overlook.,https://www.mtbproject.com/trail/6401236/zion-...,4,Featured Ride,1,0,Utah,15-20
8186,Jug Handle Loop,Moab,Blue,37.5,3247,-3249,3.8,38.5465,-109.6480,"A huge, gorgeous desert expedition that is a g...",https://www.mtbproject.com/trail/4677869/jug-h...,3,Featured Ride,1,0,Utah,30-50
8182,Mag 7,Moab,Black,21.9,1815,-3849,4.6,38.5733,-109.7924,"Long distance, ledgy, fast and fun; a premiere...",https://www.mtbproject.com/trail/652241/mag-7,5,Featured Ride,1,0,Utah,20-25
8185,Hey Joe - Spring Canyon Loop,Moab,Blue/Black,37.6,2510,-2495,4.0,38.6426,-109.8214,"A big, unlikely desert adventure to an abandon...",https://www.mtbproject.com/trail/4683120/hey-j...,4,Featured Ride,1,0,Utah,30-50


In [46]:
cos_sim_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X, n = 5, desired_state = 'Virginia')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Mount Falcon and Lair O' the Bear Loop,Morrison,Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,https://www.mtbproject.com/trail/700918/mount-...,4,Featured Ride,1,0,Colorado,15-20
8663,Braley's Pond & Confederate Breastworks Loop,Weyers Cave,Blue/Black,18.0,2753,-2758,4.0,38.2866,-79.3009,The first 1/3 of the ride is fireroad and pave...,https://www.mtbproject.com/trail/4736802/brale...,4,Featured Ride,1,0,Virginia,15-20
7893,Dry Run/Peter's Ridge Trail Loop,Covington,Blue/Black,14.6,2796,-2789,3.5,37.8030,-79.9352,Backcountry ride with a mix of gravel/Jeep roa...,https://www.mtbproject.com/trail/7025388/dry-r...,4,Featured Ride,1,0,Virginia,10-15
7053,Potts Mountain Jeep Trail Loop,New Castle,Blue/Black,18.1,2475,-2472,3.0,37.6051,-80.0758,A scenic route that circumnavigates Barbours C...,https://www.mtbproject.com/trail/7017884/potts...,4,Featured Ride,1,0,Virginia,15-20
5900,Rush Creek Loop,Chilhowie,Blue/Black,13.7,2461,-2451,4.0,36.6552,-81.6902,A variety of trails in the Rush Creek area of ...,https://www.mtbproject.com/trail/4861590/rush-...,4,Featured Ride,1,0,Virginia,10-15
7894,Blue Suck Falls to Fore Mountain (South),Clifton Forge,Blue,16.5,2538,-2542,3.5,37.9130,-79.7974,A direct ride out of Douthat State Park to the...,https://www.mtbproject.com/trail/7000244/blue-...,3,Featured Ride,1,0,Virginia,15-20


In [47]:
cos_sim_recommendations("The Whole Enchilada", 'Utah', Big_X, n = 15)

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,The Whole Enchilada,Moab,Black,34.1,1283,-7794,4.8,38.4855,-109.2320,"One of the world's great rides, with 8000 ft o...",https://www.mtbproject.com/trail/4670265/the-w...,5,Featured Ride,1,0,Utah,30-50
4551,Cannell Trail - IMBA EPIC,Kernville,Black,25.5,1718,-8030,4.6,35.9905,-118.3657,This epic Southern California jewel is a true ...,https://www.mtbproject.com/trail/5746950/canne...,5,Featured Ride,1,0,California,25-30
2933,Bike For Bender Hell Ride,Big Bear City,Black,35.9,2349,-7055,4.8,34.1707,-116.8299,A point-to-point adventure from South Fork Cam...,https://www.mtbproject.com/trail/7032168/bike-...,5,Featured Ride,1,0,California,30-50
8132,Never Eat Soggy Waffels,Junction,Black,22.3,1778,-6709,4.0,38.3790,-112.4002,Shuttled ride from mud lake down to Beaver lin...,https://www.mtbproject.com/trail/7030653/never...,5,Featured Ride,1,0,Utah,20-25
10089,Big Boulder from Packer Saddle,Downieville,Black,21.8,1644,-5813,4.3,39.6145,-120.6666,You still have about 2000ft of climbing on you...,https://www.mtbproject.com/trail/7008693/big-b...,5,Featured Ride,1,0,California,20-25
1332,CDO,Catalina,Black,21.1,1105,-6939,3.6,32.4407,-110.7861,An amazing trail traversing from the top of Mt...,https://www.mtbproject.com/trail/654291/cdo,5,Featured Ride,1,0,Arizona,20-25
8361,Monarch Crest - IMBA EPIC,Whitepine,Blue/Black,36.2,2035,-5890,4.7,38.4965,-106.3254,One of Colorado's top 5 epic rides - classic h...,https://www.mtbproject.com/trail/3671983/monar...,4,Featured Ride,1,0,Colorado,30-50
1088,Lemmon Drop,Tanque Verde,Black,18.8,1928,-6457,4.1,32.3980,-110.6893,"A long, mostly downhill tour of Mt. Lemmon.",https://www.mtbproject.com/trail/303006/lemmon...,5,Featured Ride,1,0,Arizona,15-20
15995,Paulina Peak to Horse Butte,La Pine,Blue/Black,38.7,1878,-5842,4.0,43.6895,-121.2545,An amazing tour of the Newberry Crater with a ...,https://www.mtbproject.com/trail/7018527/pauli...,4,Featured Ride,1,0,Oregon,30-50
12,Haleakala Shuttle,Kēōkea,Black,17.8,186,-7043,5.0,20.7086,-156.2540,An epic descent from the rim of the Haleakala ...,https://www.mtbproject.com/trail/7021118/halea...,5,Featured Ride,1,0,Hawaii,15-20


In [48]:
cos_sim_recommendations("Porcupine Rim", 'Utah', Big_X, n = 15, desired_state = 'Colorado', desired_location = 'Crested Butte')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Porcupine Rim,Moab,Black,14.7,1195,-2962,4.7,38.5819,-109.4164,The world famous Porcupine Rim ride; a bone ja...,https://www.mtbproject.com/trail/157369/porcup...,5,Featured Ride,1,0,Utah,10-15
9269,401 Trail Loop,Crested Butte,Blue/Black,14.1,2249,-2222,4.6,38.9661,-106.9937,The Crested Butte mega classic! High alpine me...,https://www.mtbproject.com/trail/338027/401-tr...,4,Featured Ride,1,0,Colorado,10-15
9270,Teocalli Ridge Loop,Crested Butte,Blue/Black,11.5,2184,-2181,4.6,38.8969,-106.8852,"A tough climb, great views, and a fun, long do...",https://www.mtbproject.com/trail/963897/teocal...,4,Featured Ride,1,0,Colorado,10-15
9283,Dyke Trail Loop,Crested Butte,Blue/Black,13.9,2265,-2270,3.8,38.8582,-107.0965,A Crested Butte classic with a bit of everything.,https://www.mtbproject.com/trail/963508/dyke-t...,4,Featured Ride,1,0,Colorado,10-15
9282,409s and Point Lookout Loop,Crested Butte,Blue/Black,16.9,2708,-2709,4.2,38.8644,-106.9130,"Great views, a fun, moderately technical desce...",https://www.mtbproject.com/trail/6310287/409s-...,4,Featured Ride,1,0,Colorado,15-20
9285,Green Lake Trail,Crested Butte,Black,8.4,1684,-1683,3.6,38.8671,-106.9875,One of the best out and back rides in Crested ...,https://www.mtbproject.com/trail/1222409/green...,5,Featured Ride,1,0,Colorado,5-10
9280,"Crystal Peak, Star Pass, Hunters Creek Loop",Crested Butte,Blue/Black,19.8,3265,-3268,4.6,38.8593,-106.8051,"High alpine riding, fast flowy downhill and fa...",https://www.mtbproject.com/trail/7004599/cryst...,4,Featured Ride,1,0,Colorado,15-20
9271,Reno/Flag/Bear/Deadman's Gulch,Crested Butte,Blue/Black,18.3,3402,-3415,4.3,38.8593,-106.8050,A Crested Butte classic with 3 distinct uphill...,https://www.mtbproject.com/trail/651609/renofl...,4,Featured Ride,1,0,Colorado,15-20
9275,"Snodgrass, Lupine, Lower Loop Combo",Crested Butte,Blue,22.2,2199,-2194,4.4,38.8698,-106.9805,A classic intermediate loop close to town.,https://www.mtbproject.com/trail/7003580/snodg...,3,Featured Ride,1,0,Colorado,20-25
9288,Deer Creek Loop,Crested Butte,Blue/Black,28.9,3132,-3135,3.3,38.8698,-106.9807,A long cross-country ride with stellar views a...,https://www.mtbproject.com/trail/7004590/deer-...,4,Featured Ride,1,0,Colorado,25-30


In [49]:
cos_sim_recommendations("Porcupine Rim", 'Utah', Big_X, n = 15, desired_state = 'South Carolina')

,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,url,difficulty_encoded,category,type_Featured Ride,type_Trail,state,length_range
original,Porcupine Rim,Moab,Black,14.7,1195,-2962,4.7,38.5819,-109.4164,The world famous Porcupine Rim ride; a bone ja...,https://www.mtbproject.com/trail/157369/porcup...,5,Featured Ride,1,0,Utah,10-15
4340,Paris Mountain State Park,Wade Hampton,Blue/Black,11.5,1534,-1534,4.5,34.9265,-82.3691,A fun ride that showcases the great terrain an...,https://www.mtbproject.com/trail/7000059/paris...,4,Featured Ride,1,0,South Carolina,10-15
3772,Tiger Rag Loop,Clemson,Blue/Black,9.6,960,-954,4.3,34.7471,-82.8663,A fast loop that is harder than it seems,https://www.mtbproject.com/trail/4309815/tiger...,4,Featured Ride,1,0,South Carolina,5-10
3774,20 Miles of Northern Issaqueena,Clemson,Blue,19.6,2113,-2114,3.8,34.7383,-82.8414,"The faster side of Issaqueena, connecting a nu...",https://www.mtbproject.com/trail/7011469/20-mi...,3,Featured Ride,1,0,South Carolina,15-20
3775,The Bottom End,Clemson,Blue,21.9,2281,-2283,3.7,34.7169,-82.8394,This ride includes all trails in the lower end...,https://www.mtbproject.com/trail/7011960/the-b...,3,Featured Ride,1,0,South Carolina,20-25
3773,Tour of Issaqueena,Clemson,Blue,10.3,1131,-1130,4.0,34.7384,-82.8411,This covers most of the popular trails in Issa...,https://www.mtbproject.com/trail/4706494/tour-...,3,Featured Ride,1,0,South Carolina,10-15
3776,Fant's Grove Perimeter Loop,Pendleton,Blue,20.3,1985,-1985,3.2,34.6353,-82.8129,A ride around the perimeter of Fant's Grove us...,https://www.mtbproject.com/trail/7023181/fants...,3,Featured Ride,1,0,South Carolina,20-25
2522,FATS: Brown Wave & Skinny,Murphys Estates,Blue,12.2,927,-925,4.5,33.6082,-82.0799,Super fast loops of Brown Wave and Skinny,https://www.mtbproject.com/trail/5129672/fats-...,3,Featured Ride,1,0,South Carolina,10-15
3777,NICA RIDE,Clemson,Blue,16.1,1912,-1916,3.0,34.7388,-82.8404,Upstate Shredders!,https://www.mtbproject.com/trail/7031884/nica-...,3,Featured Ride,1,0,South Carolina,15-20
1951,Marrington Trails,Goose Creek,Green/Blue,17.3,2014,-2020,3.9,32.9633,-79.9565,If you feel the need for speed this is the place!,https://www.mtbproject.com/trail/7003390/marri...,2,Featured Ride,1,0,South Carolina,15-20
